In [103]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import re
import time
import numpy as np

In [131]:
def scrape_articles(journal, start, end, times, keywords):
    
    email = 'xianlin.ding@sciencespo.fr'
    password = '099115Keep12?'
    
    # go into the Eruopresse website
    url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.find_element(By.ID, 'username').send_keys(email)
    driver.find_element(By.ID, 'password').send_keys(password)
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, 'btn-submit').click()
    
    # choose the criteria of searching
    driver.find_element(By.CLASS_NAME, 'lnk-text').click()
    time.sleep(3)
    driver.find_element(By.ID, 'specific-sources-rd').click()
    time.sleep(10)
    
    # choose the journal
    driver.find_element(By.ID, journal).click()
    
    # key words
    driver.find_element(By.XPATH, '//textarea').send_keys(keywords) 
    
    # choose the searching period
    Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

    Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value(start[0])
    Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value(end[0])

    Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value(start[1])
    Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value(end[1])

    Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value(start[2])
    Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value(end[2])
    
    driver.find_element(By.ID, 'btnSearch').click()
    
    # scroll page
    for i in range(times):
        temp_height=0
        while True:
            driver.execute_script("window.scrollBy(0,1000)")
            check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
            if check_height==temp_height:
                break
            temp_height=check_height
        
        time.sleep(5)
        if (i+1) % 5 == 0:
            print('scrolling ', i+1, ' times' )
        
    # get web contents
    web_content = driver.page_source
    obj = BeautifulSoup(web_content,'lxml').body
    obj = obj.aside.ul
    obj = obj.find_all(name='div', attrs={"class":"docListItem msDocItem"})
    
    # start to build data
    info = pd.DataFrame(columns = ['date', 'publication', 
                                   'words', 'author',
                                   'title', 'intro'])
    
    i=0
    for obj_i in obj:
        
        obj_i = obj_i.div.next_sibling.next_sibling
        
        # publication
        obj_i_1 = obj_i.div
        obj_i_1 = obj_i_1.find_all(name='span', attrs={"class":"source-name"})[0]
        obj_i_1 = str(obj_i_1)
        publication_i = re.findall(r"<span class=\"source-name\">(.+?)</span>", obj_i_1)[0]
        
        # title
        obj_i_2 = obj_i.div.next_sibling.next_sibling
        title_i = obj_i_2.div.a
        title_i = str(title_i)
        title_i = re.findall(r">(.+?)</a>", title_i)[0]
        title_i = re.sub('\u200a','',title_i)
        
        # date and words
        obj_i_3 = obj_i_2.div.next_sibling.next_sibling 
        
        date_words_i = obj_i_3.div
        date_words_i = str(date_words_i)
        date_i = re.findall(r"<span class=\"details\">(.+?)<span class=", date_words_i)[0]
        date_i = date_i.replace(' ', '')
        
        words_i = re.findall(r"</span>(.+?)</span>", date_words_i)[0]
        words_i = words_i.replace(' ', '')
        
        # authors and introduction
        obj_i_4 = obj_i_3.div.next_sibling.next_sibling
        obj_i_4 = str(obj_i_4)
        
        try:
            obj_i_4 = obj_i_4.split('<span class="doclist-author">')[1]
            author_i = obj_i_4.split('</span>')[0]
            intro_i = obj_i_4.split('</span>')[1]
        
        except:
            author_i = None
            intro_i = obj_i_4.split('</span> </span>')[1]
    
        intro_i = intro_i.split('</div>')[0]
        intro_i = re.sub(r"<([a-z]+)>",'',intro_i)
        intro_i = re.sub(r"</([a-z]+)>",'',intro_i)
        intro_i = re.sub('\u200a','',intro_i)
        intro_i = re.sub('-','',intro_i)
        
        info.loc[i] = [date_i, publication_i, 
                       words_i, author_i,
                       title_i, intro_i]
        i = i+1
    
    return info

In [132]:
import os

path = os.getcwd()
path = re.sub('Code','Result',path)

In [133]:
## data 1 : keywords - climat

journal = 'sf_247'  # le monde
keywords = 'climat'  # keywords
start = ['2022','9','1'] # searching period
end = ['2022','12','31']
times = 30 # scrolling times

data_1 = scrape_articles(journal, start, end, times, keywords)

<ipython-input-131-3f966b9c1daf>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times


In [134]:
data_1.to_csv(path + '\\compare\\compare_1.csv')
data_1.tail(2)

,date,publication,words,author,title,intro
598,2022-12-07,Le Monde,1208mots,Ivanne Trippenbach,L’appel de Macron à ériger des « remparts » fa...,"... des incarcérations, termine par des a..."
599,2022-11-09,Le Monde,650mots,Nathaniel Herzberg,Le gorfou qui couve un œuf sur deux,"Dans le climat hostile de l’extrême Sud, ..."


In [135]:
## data 2 : keywords - climatique

journal = 'sf_247'  # le monde
keywords = 'climatique'  # keywords
start = ['2022','9','1'] # searching period
end = ['2022','12','31']
times = 30 # scrolling times

data_2 = scrape_articles(journal, start, end, times, keywords)

<ipython-input-131-3f966b9c1daf>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times


In [136]:
data_2.to_csv(path + '\\compare\\compare_2.csv')
data_2.tail(2)

,date,publication,words,author,title,intro
652,2022-09-07,Le Monde,1023mots,Delphine Roucaute,Plus de 11000 décès seraient liés aux vagues d...,"... chaleur. En creux, c’est la capacité ..."
653,2022-09-06,Le Monde,1699mots,None,Pauline Londeix et Jérôme Martin L’Etat doit r...,"... ’évolution des modes de vie, l’imp..."


In [112]:
## data 3 : keywords - changement climatique

journal = 'sf_247'  # le monde
keywords = 'changement climatique'  # keywords
start = ['2022','9','1'] # searching period
end = ['2022','12','31']
times = 30 # scrolling times

data_3 = scrape_articles(journal, start, end, times, keywords)

<ipython-input-104-c1161130a85c>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times


In [128]:
data_3.to_csv(path + '\\compare\\compare_3.csv')
data_3.tail(2)

,date,publication,words,author,title,intro
270,2022-09-06,Le Monde,1699mots,None,Pauline Londeix et Jérôme Martin L’Etat doit r...,"... L’évolution des modes de vie, l’i..."
271,2022-09-03,Le Monde,569mots,Harry Bernas,Harry Bernas La centrale nucléaire de Zaporiji...,... dirigeants reviennent au discours des...


In [42]:
## data 4 : keywords - réchauffement

journal = 'sf_247'  # le monde
keywords = 'réchauffement'  # keywords
start = ['2022','9','1'] # searching period
end = ['2022','12','31']
times = 30 # scrolling times

data_4 = scrape_articles(journal, start, end, times, keywords)

scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times


In [129]:
data_4.to_csv(path + '\\compare\\compare_4.csv')
data_4.tail(2)

,date,publication,words,author,title,intro
200,2022-09-03,Le Monde,569mots,Harry Bernas,Harry Bernas La centrale nucléaire de Zaporiji...,... des combustibles usagés. Quelles leço...
201,2022-09-02,Le Monde,798mots,Perrine Mouterde,Des gendarmes bientôt formés aux atteintes à l...,... Les pistes pour mieux prévenir les in...


In [46]:
## data 5 : keywords - effet de serre

journal = 'sf_247'  # le monde
keywords = 'effet de serre'  # keywords
start = ['2022','9','1'] # searching period
end = ['2022','12','31']
times = 30 # scrolling times

data_5 = scrape_articles(journal, start, end, times, keywords)

scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times


In [47]:
data_5.to_csv(path + '\\compare_5.csv')
data_5.tail(2)

,date,publication,words,author,title,intro
200,2022-09-03,Le Monde,569mots,Harry Bernas,Harry Bernas La centrale nucléaire de Zaporiji...,... des combustibles usagés. Quelles leço...
201,2022-09-02,Le Monde,798mots,Perrine Mouterde,Des gendarmes bientôt formés aux atteintes à l...,... Les pistes pour mieux prévenir les in...


In [115]:
# compare data_1 and data_2
# data_2 : 654 articles, 41.28%
data_1_2 = pd.concat([data_1,data_2], axis=0)
data_1_2 = data_1_2.reset_index(drop=True)
data_1_2 = data_1_2[['date', 'publication', 'words', 'title']]

pd.concat([data_1_2.drop_duplicates(),
           data_1_2.drop_duplicates(keep=False)]).drop_duplicates(keep=False).reset_index(drop=True).tail(2)

,date,publication,words,title
268,2022-09-06,Le Monde,994mots,"Trois soirées, huit possibilités"
269,2022-09-09,Le Monde,1022mots,Bruno Palier Miser sur une protection sociale ...


In [116]:
# compare data_1 and data_3
# data_3: 325 articles, 50%
data_1_3 = pd.concat([data_1,data_3], axis=0)
data_1_3 = data_1_3.reset_index(drop=True)
data_1_3 = data_1_3[['date', 'publication', 'words', 'title']]

duplicates_1_3 =  pd.concat([data_1_3.drop_duplicates(),
                  data_1_3.drop_duplicates(keep=False)]).drop_duplicates(keep=False).reset_index(drop=True)
duplicates_1_3.tail(2)

,date,publication,words,title
134,2022-09-01,Le Monde,192mots,Climat Le deuxième été le plus chaud en France
135,2022-09-06,Le Monde,2266mots,"Actualité, enjeux de société: on débat!"


In [120]:
pd.concat([data_3[['date', 'publication', 'words', 'title']],
           duplicates_1_3]).drop_duplicates(keep=False).reset_index(drop=True)['title']

0      Le changement climatique pourrait faire monter...
1           Le Pakistan pâtit d’infrastructures vétustes
2          Le choléra se diffuse et les vaccins manquent
3         Le sable, richesse et malédiction des Maldives
4      Les moussons menacent la cité antique de Mohen...
                             ...                        
131      Le Pakistan ravagé par le changement climatique
132          La gestion des forêts appelée à se réformer
133                             De l’or noir à l’or vert
134    Pauline Londeix et Jérôme Martin L’Etat doit r...
135    Harry Bernas La centrale nucléaire de Zaporiji...
Name: title, Length: 136, dtype: object

In [121]:
# compare data_2 and data_3
# data_3: 325 articles, 50%
data_2_3 = pd.concat([data_2,data_3], axis=0)
data_2_3 = data_2_3.reset_index(drop=True)
data_2_3 = data_2_3[['date', 'publication', 'words', 'title']]

duplicates_2_3 =  pd.concat([data_2_3.drop_duplicates(),
                  data_2_3.drop_duplicates(keep=False)]).drop_duplicates(keep=False).reset_index(drop=True)
duplicates_2_3.tail(2)

,date,publication,words,title
270,2022-09-05,Le Monde,905mots,Céline Azemar Comment taxer les superprofits d...
271,2022-09-06,Le Monde,1699mots,Pauline Londeix et Jérôme Martin L’Etat doit r...


In [118]:
# compare data_1 and data_4
# data_4: 201 articles, 60%
data_1_4 = pd.concat([data_1,data_4], axis=0)
data_1_4 = data_1_4.reset_index(drop=True)
data_1_4 = data_1_4[['date', 'publication', 'words', 'title']]

duplicates_1_4 = pd.concat([data_1_4.drop_duplicates(),
                 data_1_4.drop_duplicates(keep=False)]).drop_duplicates(keep=False).reset_index(drop=True)
duplicates_1_4.tail(2)

,date,publication,words,title
83,2022-09-01,Le Monde,1136mots,Le deuxième été le plus chaud en France
84,2022-09-01,Le Monde,192mots,Climat Le deuxième été le plus chaud en France


In [119]:
pd.concat([data_4[['date', 'publication', 'words', 'title']],
           duplicates_1_4]).drop_duplicates(keep=False).reset_index(drop=True).tail(2)

,date,publication,words,title
115,2022-09-03,Le Monde,569mots,Harry Bernas La centrale nucléaire de Zaporiji...
116,2022-09-02,Le Monde,798mots,Des gendarmes bientôt formés aux atteintes à l...


In [124]:
list(data_1['title'])

['Antonio Guterres, voix ardente pour le climat',
 '«Face au réchauffement, chaque Etat doit sortir de sa zone de confort»',
 'L’agriculture et l’alimentation oubliées',
 'Valérie Masson-Delmotte, une climatologue au sommet',
 'Pour le climat, «une année gâchée»',
 'Climat : les promesses non tenues du budget 2023',
 'Une COP pour garder le climat au premier plan',
 'Guterres plaide pour taxer les profits dans les énergies fossiles',
 'Macron et la ligne de crête de l’écologie',
 'Pub : une mue contre-nature',
 'Climat et biodiversité : deux crises liées',
 'COP27 : des progrès ténus',
 'COP27 : Lula promet un « leadership » sur le climat',
 'Allemagne : Lützerath, un village au bord de la mine',
 "En Suède, l'extrême retournement d'une jeune libérale.",
 'L’UE s’accorde sur la réforme du marché carbone',
 'Climat: des seuils critiques bientôt franchis?',
 'Le sursaut «fragile» des investissements pour le climat en France',
 '« Les solutions doivent être convergentes »',
 'Le succès de